In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
from matplotlib import pyplot as plt
import sys
import time

sys.path.append('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/')

from scripts.utils import create_one_hot_encoding

pd.set_option('display.max_columns', 500)

import warnings
warnings.filterwarnings("ignore")

In [2]:
df_transactions = pd.read_parquet('../../data/compressed_dataset/transactions.parquet')
df_articles = pd.read_parquet('../../data/compressed_dataset/articles.parquet')
df_customers = pd.read_parquet('../../data/compressed_dataset/customers.parquet')

article_id_int = pd.read_pickle('../../data/compressed_dataset/article_id_int.pickle')
int_article_id = pd.read_pickle('../../data/compressed_dataset/int_article_id.pickle')

customer_id_int = pd.read_pickle('../../data/compressed_dataset/customer_id_int.pickle')
int_customer_id = pd.read_pickle('../../data/compressed_dataset/int_customer_id.pickle')

# Days from first purchase, date from last purchase

In [3]:
df_transactions['t_dat'] = pd.to_datetime(df_transactions['t_dat'])
last_trans_date = df_transactions['t_dat'].max()


num_train_weeks = 20

for i in range(num_train_weeks-1, -1, -1):
    
    candidates_predictions = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/ranker_train_set_utils/models_preds_{}.parquet'.format(i))['article_id'].unique()
        
    
    last_local_date = last_trans_date - timedelta(days=i * 7)
    
    train_set_st1 = df_transactions[df_transactions['t_dat'] <= last_local_date].copy()
    train_set_st1['weeks_before_sub'] = i
    
    train_set_st1 = train_set_st1[train_set_st1['article_id'].isin(candidates_predictions)]
        
    df_trans_first_purchase = train_set_st1.groupby(['article_id']).first().reset_index()
    df_trans_last_purchase = train_set_st1.groupby(['article_id']).last().reset_index()
    
    train_set_st1 = train_set_st1.groupby(['article_id']).first().reset_index()
    
    train_set_st1 = train_set_st1.merge(df_trans_first_purchase[['article_id', 't_dat']].rename(columns={'t_dat': 'first_dat'}),
                                      on=['article_id'])

    train_set_st1 = train_set_st1.merge(df_trans_last_purchase[['article_id', 't_dat']].rename(columns={'t_dat': 'last_dat'}),
                                          on=['article_id'])
    
    
    train_set_st1['article__num_days_from_first_purchase'] = (last_local_date - train_set_st1['first_dat']).dt.days
    train_set_st1['article__num_days_from_last_purchase'] = (last_local_date - train_set_st1['last_dat']).dt.days
    
    train_set_st1 = train_set_st1[['article_id', 'weeks_before_sub',
                                   'article__num_days_from_first_purchase', 'article__num_days_from_last_purchase']]
    
    if i == num_train_weeks-1:
        all_train_set = train_set_st1
    else:
        all_train_set = pd.concat([all_train_set, train_set_st1], ignore_index=True)
    

all_train_set.to_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/articles/dynamic/num_days_from_purchases.parquet',
                        index=False)

# mean_sales_channel_id

In [4]:
df_transactions['t_dat'] = pd.to_datetime(df_transactions['t_dat'])
last_trans_date = df_transactions['t_dat'].max()


num_train_weeks = 20

for i in range(num_train_weeks-1, -1, -1):
    
    candidates_predictions = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/ranker_train_set_utils/models_preds_{}.parquet'.format(i))['article_id'].unique()
        
    last_local_date = last_trans_date - timedelta(days=i * 7)
    
    train_set_st1 = df_transactions[df_transactions['t_dat'] <= last_local_date].copy()
    
    train_set_st1 = train_set_st1[train_set_st1['article_id'].isin(candidates_predictions)]
        
    train_set_st1 = train_set_st1.groupby(['article_id'])['sales_channel_id'].mean().reset_index().rename(columns={'sales_channel_id': 'article__mean_sales_channel_id'})
    train_set_st1['weeks_before_sub'] = i
        
    train_set_st1 = train_set_st1[['article_id', 'weeks_before_sub',
                                   'article__mean_sales_channel_id']]
    
    if i == num_train_weeks-1:
        all_train_set = train_set_st1
    else:
        all_train_set = pd.concat([all_train_set, train_set_st1], ignore_index=True)
    

all_train_set.to_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/articles/dynamic/mean_sales_channel_id.parquet',
                        index=False)

# mean_price

In [5]:
df_transactions['t_dat'] = pd.to_datetime(df_transactions['t_dat'])
last_trans_date = df_transactions['t_dat'].max()


num_train_weeks = 20

for i in range(num_train_weeks-1, -1, -1):
    
    candidates_predictions = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/ranker_train_set_utils/models_preds_{}.parquet'.format(i))['article_id'].unique()
        
    
    last_local_date = last_trans_date - timedelta(days=i * 7)
    
    train_set_st1 = df_transactions[df_transactions['t_dat'] <= last_local_date].copy()
    
    train_set_st1 = train_set_st1[train_set_st1['article_id'].isin(candidates_predictions)]
    
    train_set_last = train_set_st1.groupby(['article_id'])['price'].last().reset_index().rename(columns={'price': 'article__last_price'})
    
    train_set_st1 = train_set_st1.groupby(['article_id'])['price'].mean().reset_index().rename(columns={'price': 'article__mean_price'})
        
    train_set_st1['weeks_before_sub'] = i
        
    train_set_st1 = train_set_st1[['article_id', 'weeks_before_sub',
                                   'article__mean_price']]
    
    train_set_st1 = train_set_st1.merge(train_set_last[['article_id', 'article__last_price']], how='left', on=['article_id'])
    train_set_st1['article__last_price_ratio'] = train_set_st1['article__last_price'] / train_set_st1['article__mean_price']
    
    if i == num_train_weeks-1:
        all_train_set = train_set_st1
    else:
        all_train_set = pd.concat([all_train_set, train_set_st1], ignore_index=True)
    

all_train_set.to_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/articles/dynamic/price.parquet',
                        index=False)

# num purchased for all time / last 30d / last 7d

In [8]:
df_transactions['t_dat'] = pd.to_datetime(df_transactions['t_dat'])
last_trans_date = df_transactions['t_dat'].max()


num_train_weeks = 20

for i in range(num_train_weeks-1, -1, -1):
    
    candidates_predictions = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/ranker_train_set_utils/models_preds_{}.parquet'.format(i))['article_id'].unique()
        
    
    last_local_date = last_trans_date - timedelta(days=i * 7)
    
    train_set_st1 = df_transactions[df_transactions['t_dat'] <= last_local_date].copy()
    
    train_set_st1 = train_set_st1[train_set_st1['article_id'].isin(candidates_predictions)]
        
    train_set_st1_temp = train_set_st1.groupby(['article_id'])['customer_id'].count().reset_index().rename(columns={'customer_id': 'article__num_purchased_customers'})
    train_set_st1_temp = train_set_st1_temp.merge(train_set_st1.groupby(['article_id'])['customer_id'].nunique().reset_index().rename(columns={'customer_id': 'article__num_unique_purchased_customers'}),
                                        on=['article_id'], how='left')
    
    train_set_st1_temp = train_set_st1_temp.merge(train_set_st1[train_set_st1['t_dat'] >= last_local_date-timedelta(days=90)].groupby(['article_id'])['customer_id'].count().reset_index().rename(columns={'customer_id': 'article__num_purchased_customers_last90days'}),
                                        on=['article_id'], how='left')
    train_set_st1_temp = train_set_st1_temp.merge(train_set_st1[train_set_st1['t_dat'] >= last_local_date-timedelta(days=90)].groupby(['article_id'])['customer_id'].nunique().reset_index().rename(columns={'customer_id': 'article__num_unique_purchased_customers_last90days'}),
                                        on=['article_id'], how='left')
    
    train_set_st1_temp = train_set_st1_temp.merge(train_set_st1[train_set_st1['t_dat'] >= last_local_date-timedelta(days=30)].groupby(['article_id'])['customer_id'].count().reset_index().rename(columns={'customer_id': 'article__num_purchased_customers_last30days'}),
                                        on=['article_id'], how='left')
    train_set_st1_temp = train_set_st1_temp.merge(train_set_st1[train_set_st1['t_dat'] >= last_local_date-timedelta(days=30)].groupby(['article_id'])['customer_id'].nunique().reset_index().rename(columns={'customer_id': 'article__num_unique_purchased_customers_last30days'}),
                                        on=['article_id'], how='left')
    
    
    train_set_st1_temp = train_set_st1_temp.merge(train_set_st1[train_set_st1['t_dat'] >= last_local_date-timedelta(days=7)].groupby(['article_id'])['customer_id'].count().reset_index().rename(columns={'customer_id': 'article__num_purchased_customers_last7days'}),
                                        on=['article_id'], how='left')
    train_set_st1_temp = train_set_st1_temp.merge(train_set_st1[train_set_st1['t_dat'] >= last_local_date-timedelta(days=7)].groupby(['article_id'])['customer_id'].nunique().reset_index().rename(columns={'customer_id': 'article__num_unique_purchased_customers_last7days'}),
                                        on=['article_id'], how='left')
    
    train_set_st1_temp = train_set_st1_temp.merge(train_set_st1[train_set_st1['t_dat'] == last_local_date-timedelta(days=7)].groupby(['article_id'])['customer_id'].count().reset_index().rename(columns={'customer_id': 'article__num_purchased_customers_last1days'}),
                                        on=['article_id'], how='left')
    train_set_st1_temp = train_set_st1_temp.merge(train_set_st1[train_set_st1['t_dat'] == last_local_date-timedelta(days=7)].groupby(['article_id'])['customer_id'].nunique().reset_index().rename(columns={'customer_id': 'article__num_unique_purchased_customers_last1days'}),
                                        on=['article_id'], how='left')
    
    train_set_st1_temp['article__unique_ratio'] = train_set_st1_temp['article__num_unique_purchased_customers'] / train_set_st1_temp['article__num_purchased_customers']
    train_set_st1_temp['article__unique_ratio_last90days'] = train_set_st1_temp['article__num_unique_purchased_customers_last90days'] / train_set_st1_temp['article__num_purchased_customers_last90days']
    train_set_st1_temp['article__unique_ratio_last30days'] = train_set_st1_temp['article__num_unique_purchased_customers_last30days'] / train_set_st1_temp['article__num_purchased_customers_last30days']
    train_set_st1_temp['article__unique_ratio_last7days'] = train_set_st1_temp['article__num_unique_purchased_customers_last7days'] / train_set_st1_temp['article__num_purchased_customers_last7days']
    train_set_st1_temp['article__unique_ratio_last1days'] = train_set_st1_temp['article__num_unique_purchased_customers_last1days'] / train_set_st1_temp['article__num_purchased_customers_last1days']

    
    train_set_st1_temp['weeks_before_sub'] = i
        
    
        
    train_set_st1_temp = train_set_st1_temp[['article_id', 'weeks_before_sub',
                                   'article__num_purchased_customers', 'article__num_unique_purchased_customers',
                                   'article__num_purchased_customers_last90days', 'article__num_unique_purchased_customers_last90days',
                                   'article__num_purchased_customers_last30days', 'article__num_unique_purchased_customers_last30days',
                                   'article__num_purchased_customers_last7days', 'article__num_unique_purchased_customers_last7days',
                                   'article__num_purchased_customers_last1days', 'article__num_unique_purchased_customers_last1days',
                                   'article__unique_ratio', 'article__unique_ratio_last30days', 'article__unique_ratio_last7days', 'article__unique_ratio_last1days'
                                  ]]
    
    if i == num_train_weeks-1:
        all_train_set = train_set_st1_temp
    else:
        all_train_set = pd.concat([all_train_set, train_set_st1_temp], ignore_index=True)
    
all_train_set.fillna(0, inplace=True)
all_train_set.to_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/articles/dynamic/num_purchases__ratios.parquet',
                        index=False)

In [10]:
del all_train_set

# Other features

In [11]:
num_days = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/articles/dynamic/num_days_from_purchases.parquet')
num_purchases = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/articles/dynamic/num_purchases__ratios.parquet')

In [15]:
feats = num_days.merge(num_purchases, how='left', on=['article_id', 'weeks_before_sub'])

In [17]:
feats['article__frequency_purchases'] = feats['article__num_days_from_first_purchase'] / feats['article__num_purchased_customers']

In [18]:
feats[['article_id', 'weeks_before_sub', 'article__frequency_purchases']].to_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/articles/dynamic/article__frequency_purchases.parquet',
                                                                                    index=False)